In [ ]:
import csv

import dtreeviz as dtreeviz
import numpy as np
from six import StringIO
from IPython.display import Image
from sklearn.metrics import accuracy_score
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn import metrics 
import pydotplus
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
from csv import reader
import pandas as pd
import os

In [ ]:
preprocessing()
bank_df = pd.read_csv("data/Clean_Demographic.csv")

#countplot
sns.countplot(x='default payment in Jan 2022', data=bank_df, palette='hls')
plt.xlabel('Default',fontsize=12)
plt.ylabel('Count',fontsize=13)
plt.show()
#data is skewed toward non defaulted customers

corr = bank_df.corr()
ax1 = sns.heatmap(corr, fmt='' ,annot=True, square=True)
plt.show()
#Heatmap shows there is no correlation between numerical values in the dataset

df_heatmap = bank_df.pivot_table(values='default payment in Jan 2022',index='EDUCATION',columns='GENDER',aggfunc=np.mean)
sns.heatmap(df_heatmap,annot=True)
plt.show()

feature_cols = ['ACCOUNT_TYPE', 'GENDER', 'MARITAL_STATUS', 'EDUCATION','JOB_TYPE','HOUSING','LOAN']

#make categorical data readable for sklearn, so convert it to integer
bank_df2=bank_df.drop(['ID'],axis=1)
bank_df_dummies= pd.get_dummies(bank_df2, columns=feature_cols)

print('check the column values: ',bank_df_dummies.columns.tolist())

#split dataset in features and target variable

X=bank_df_dummies.drop(['default payment in Jan 2022'],axis=1)
feature_list=X.columns.tolist()
Y=bank_df_dummies[['default payment in Jan 2022']]

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, stratify=Y) # 70% training and 30% test

# Create Decision Tree classifer object, for optimizingg we chose max depth and entropy
clf = DecisionTreeClassifier(criterion="gini", max_depth=5)

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Accuracy, how often is the classifier correct?
dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,
                filled=True, rounded=True,
                special_characters=True, feature_names =feature_list ,class_names=['0','1'])

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_png('figures/DTree.png')
Image(graph.create_png())

print('Model accuracy score with criterion gini index: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))
#visualize feature importance
print(clf.feature_importances_)

#feat_importances.nlargest(20).plot(kind='barh')
(pd.Series(clf.feature_importances_, index=X.columns)
   .nlargest(10)
   .plot(kind='barh'))

plt.bar( range(len(clf.feature_importances_)), clf.feature_importances_)
#plt.xticks(range(len(clf.feature_importances_)), X.columns)
n_features = X_train.shape[1]
plt.figure(figsize=(20,20))
plt.barh(range(n_features), clf.feature_importances_, align='center')
plt.yticks(np.arange(n_features), X_train.columns.values)
plt.xlabel('Feature importance')
plt.ylabel('Feature')

plt.show()